OBTENCIÓN DE LIBRERÍAS

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


# Transformaciones Afines

## Traslación

In [ ]:
%%writefile affine.cu
#include <bits/stdc++.h>
#include <opencv2/opencv.hpp>
#include <iostream>
#include <iomanip>
#include <cuda.h>
#include <cuda_runtime.h>
#include <cuda_runtime_api.h>
#include <cassert>
#include <cmath>
#include <vector>

#define checkCudaErrors(val) check( (val), #val, __FILE__, __LINE__)

template<typename T>
void check(T err, const char* const func, const char* const file, const int line) {
  if (err != cudaSuccess) {
    std::cerr << "CUDA error at: " << file << ":" << line << std::endl;
    std::cerr << cudaGetErrorString(err) << " " << func << std::endl;
    exit(1);
  }
}


// Max Threads per block in GeForce 210
#define TxB 512

__global__
void affine_kernel(const uchar3* const inputImage,
                         uchar3* const outputImage,
                   float** d_affine_trans,
                   int numRows, int numCols  ){
    
  int i = blockIdx.x * blockDim.x + threadIdx.x;

  //TODO: Arreglar pasar la matriz de transformación

  //traslacion
  int affine[2][3] = {{1,0,200},{0,1,200}};

  //escala
  //int affine[2][3] = {{2,0,0},{0,2,0}};

  //Rotacion
  //float affine[2][3] = {{0.1542,-0.9880,0},{0.9880,0.1542,0}};

  //Inclinacion
  //int affine[2][3] = {{1,-20,0},{-20,1,0}};

  if( i < numRows*numCols){      
      //Obtención de coordenadas
      int x = i/numCols;
      int y = i%numCols;
      //nueva posicióón
      int x_n, y_n;

      //Multiplicacion
      x_n = affine[0][0]*x + affine[0][1]*y + affine[0][2];
      y_n = affine[1][0]*x + affine[1][1]*y + affine[1][2];

      //printf("(%d,%d) -> (%d,%d) \n", x,y,new_coord[0], new_coord[1]);

      __syncthreads();

      //TODO: Columna circular
      if( (x_n*numCols + y_n) < numRows*numCols ){
          outputImage[ x_n*numCols + y_n ] = inputImage[i];
      }
      

  }              
    

}


void affine_transform(const uchar3* const d_inputImage,
                            uchar3* const d_outputImage,
                      float** d_affine_trans,
                      int numRows, int numCols )
{
    

  // Dado que no importa la posicion relativa de los pixels
  // en este algoritmo, la estrategia para asignar hilos a
  // bloques y rejillas sera sencillamente la de cubrir
  // a todos los pixeles con hebras en el eje X
  long long int total_px = numRows * numCols;  // total pixels
  long int grids_n = ceil(total_px / TxB); // grids numer
  const dim3 blockSize(TxB, 1, 1);
  const dim3 gridSize(grids_n, 1, 1);
  affine_kernel<<<gridSize, blockSize>>>(d_inputImage, d_outputImage,d_affine_trans, numRows, numCols);
  
  cudaDeviceSynchronize(); 
  checkCudaErrors(cudaGetLastError());

}

int main(){
    
    uchar3  *h_inputImage,   *d_inputImage;
    uchar3  *h_outputImage,  *d_outputImage;
    float ** d_affine_trans;
    //traslación
    float h_affine_trans[2][3] = {{1.0,0.0,2.0},
                                  {0.0,1.0,2.0}};

    //Imagen de entrada
    std::string inputImageFile = "guy.png";
    std::string outputImageFile = "guy_affine.png";

    //PREPROCESADO DE LA IMAGEN

    //Matrices de las imagenes
    cv::Mat inputImageMat;
    cv::Mat outputImageMat;

    //Punteros hacia el inicio de la imagen
    //Necesarios para luego liberar memoria
    uchar3        *d_inputImage__;
    uchar3        *d_outputImage__;


    //Comprobar que el contexto se inicializa bien
    checkCudaErrors(cudaFree(0));

    inputImageMat = cv::imread(inputImageFile.c_str(), CV_LOAD_IMAGE_COLOR);
    
    if (inputImageMat.empty()) {
      std::cerr << "No se pudo abrir el archivo: " << inputImageFile << std::endl;
      exit(1);
    }


    // Reserva memoria para el output
    outputImageMat.create(inputImageMat.rows, inputImageMat.cols, CV_8UC3);

    //This shouldn't ever happen given the way the images are created
    //at least based upon my limited understanding of OpenCV, but better to check
    if (!inputImageMat.isContinuous() || !outputImageMat.isContinuous()) {
      std::cerr << "Images aren't continuous!! Exiting." << std::endl;
      exit(1);
    }

    //Apuntamos al comienzo de las filas
    h_inputImage   = (uchar3 *)inputImageMat.ptr<unsigned char>(0);
    h_outputImage  = (uchar3 *)outputImageMat.ptr<unsigned char>(0);

    const size_t numPixels = inputImageMat.rows * inputImageMat.cols;
    //Reserva memoria en el dispositivo
    checkCudaErrors(cudaMalloc(&d_inputImage,  sizeof(uchar3) * numPixels));
    checkCudaErrors(cudaMalloc(&d_outputImage, sizeof(uchar3) * numPixels));
    checkCudaErrors(cudaMalloc(&d_affine_trans, sizeof(float)*2*3 ));
    checkCudaErrors(cudaMemset(d_outputImage, 0, numPixels * sizeof(uchar3))); // Asegurate de que no queda memoria sin liberar

    // Copia el input en la GPU
    checkCudaErrors(cudaMemcpy(d_inputImage, h_inputImage, sizeof(uchar3) * numPixels, cudaMemcpyHostToDevice));
    checkCudaErrors(cudaMemcpy(d_affine_trans, h_affine_trans, sizeof( sizeof(float)*2*3 ), cudaMemcpyHostToDevice));

    d_inputImage__  = d_inputImage;
    d_outputImage__ = d_outputImage;


    //LLAMADA DE LA FUNCIÓN CUDA
    affine_transform(d_inputImage, d_outputImage, d_affine_trans, inputImageMat.rows, inputImageMat.cols);


    //EXTRACCIÓN DE IMAGEN
    checkCudaErrors(cudaMemcpy(h_outputImage, d_outputImage, sizeof(uchar3) * numPixels, cudaMemcpyDeviceToHost));

    //Guardamos la imagen
    cv::Mat output(inputImageMat.rows, inputImageMat.cols, CV_8UC3, (void*)h_outputImage);
    cv::imwrite(outputImageFile.c_str(), output);

    /* Libera memoria */
    cudaFree(d_inputImage__);
    cudaFree(d_outputImage__);

    return 0;
}



Overwriting affine.cu


Ejecución del kernel

In [ ]:
!nvcc affine.cu `pkg-config --cflags --libs opencv` -o affine; ./affine